In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import thinkstats2
import math
import seaborn as sns
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.preprocessing import StandardScaler
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer
from sklearn.impute import IterativeImputer
from scipy.stats import entropy
from scipy.stats import ks_2samp
from sklearn import metrics
from eda_kit import conditional_entropy,roc_auc_score,auc

In [8]:
A_train = pd.read_csv('../data/A_train.csv')
B_train = pd.read_csv('../data/B_train_discrete.csv')
B_test = pd.read_csv('../data/B_test_discrete.csv')

In [9]:
target_bb = B_train['flag']
bb_data = B_train.drop('flag',axis=1)
bbt_data = B_test


In [10]:
#   GBDT训练 输出（47%以上的特征）   ，训练集划分交叉验证

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(bb_data, target_bb, test_size=0.3, random_state=0)
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1,  max_depth=1, random_state=0)
clf.fit(X_train, y_train)
clf.score(X_test, y_test)
y_pred = clf.predict(X_test)
y_pro = clf.predict_proba(X_test)
y_prd = pd.DataFrame(y_pro).iloc[:,1]
roc_auc_score(y_test,y_prd)

# 训练所有数据 输出

clf.fit(bb_data,target_bb)
b = clf.predict_proba(bbt_data)
b=pd.DataFrame(b)
pro_b = b.iloc[:,1]
no = B_test.iloc[:,0]
pro = pd.DataFrame(pro_b)
no = pd.DataFrame(no)


# GBDT 重要特征

clf.fit(bb_data,target_bb)
clf_importance = clf.feature_importances_
clf_importance_ = pd.DataFrame(clf_importance)
clf_importance_.columns = {'importance'}
bb_columns = pd.DataFrame(bb_data.columns)
bb_columns.columns={'feature'}

#影响度排序
clf_feature_values = pd.concat([bb_columns,clf_importance_],axis=1)
clf_feature_values = clf_feature_values.sort_values(by='importance')


#影响度非0的特征
clf_feature_well = clf_feature_values[clf_feature_values['importance']!=0]
clf_feature_well_columns = clf_feature_well['feature'].values
clf_feature_well.index = clf_feature_well_columns
columns_GBDT = clf_feature_well.index

# 测试集提取这些特征，形成新的测试集

gbdt_b_data = B_train[columns_GBDT]
gbdt_bt_data = B_test[columns_GBDT]
gbdt_b_flag = pd.DataFrame(B_train['flag'])
gbdt_b_train = pd.concat([gbdt_b_data,gbdt_b_flag],axis=1)

gbdt_fillna_b_data = gbdt_b_data.fillna(0)
new_test_  = gbdt_bt_data.fillna(0)

In [11]:
#   重要特征训练，训练集交叉验证

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(gbdt_fillna_b_data, gbdt_b_flag, test_size=0.3, random_state=0)
clf = GradientBoostingClassifier(n_estimators=110, learning_rate=1,  max_depth=1, random_state=0)#loss='exponential' mse
clf.fit(X_train, y_train)
clf.score(X_test, y_test)
y_pred = clf.predict(X_test)
y_pro = clf.predict_proba(X_test)
y_prd = pd.DataFrame(y_pro).iloc[:,1]
roc_auc_score(y_test,y_prd)
clf.fit(gbdt_fillna_b_data,gbdt_b_flag)
b = clf.predict_proba(new_test_)
b=pd.DataFrame(b)
pro_b = b.iloc[:,1]
no = B_test.iloc[:,0]
pro = pd.DataFrame(pro_b)
no = pd.DataFrame(no)

/Users/wenqiang/opt/anaconda3/envs/qhzx37/lib/python3.7/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/Users/wenqiang/opt/anaconda3/envs/qhzx37/lib/python3.7/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [12]:
import xgboost as xgb

xg_train = xgb.DMatrix(X_train,label=y_train)
xg_test = xgb.DMatrix(X_test,label=y_test)


param = {'booster':'gbtree',
         'max_depth':10,
         'eta':0.1,
         'silent':1,
         'objective':'binary:logistic',
         'eval_metric':'auc',
         'subsample': 1,
         "colsample_bytree": 0.7,
         "min_child_weight":2,
              'gamma':3.1,
              'lambda':1,
        "thread":-1,}
num_boost_round = 1500
watchlist = [(xg_train, 'train'), (xg_test, 'eval')]
num_round=15
bst = xgb.train(param, xg_train, num_round)
preds = bst.predict(xg_test)
roc_auc_score(y_test,preds)

xg_train = xgb.DMatrix(gbdt_fillna_b_data,label=gbdt_b_flag)
xg_test = xgb.DMatrix(new_test_)
bst = xgb.train(param, xg_train, num_round)
pro = bst.predict(xg_test)

xgb_model =xgb.cv(param,xg_train,num_boost_round,nfold=5, early_stopping_rounds=300)#, verbose_eval=True

pd.DataFrame(xgb_model)
# 0.5829
pd.DataFrame(xgb_model)['test-auc-mean'].mean()



# b=pd.DataFrame(b)
#
# XGB_B=pd.DataFrame([])
# XGB_B['no'] = no['no']
# XGB_B['pred'] = pro

0.5516511538461539